##### This notebook will be mainly used for the Capstone Project in the IBM Data Science Certification courses.

# Part 1: Web Scape the Wiki page for Toronto Neighborhoods

In [3]:
import sys
!{sys.executable} -m pip install beautifulsoup4

     |████████████████████████████████| 122kB 5.3MB/s eta 0:00:01


In [2]:
# importing the libraries
from bs4 import BeautifulSoup
import requests

# Step 1: Sending a HTTP request to a URL
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

In [3]:
# Step 2: Parse the html content
soup = BeautifulSoup(html_content, "html.parser")
#print(soup.prettify()) # print the parsed data of html

In [4]:
# Step 3: Get the table having the class wikitable sortable
gdp_table = soup.find("table", attrs={"class": "wikitable sortable"})
gdp_table_data = gdp_table.tbody.find_all("tr")  # contains 2 rows

In [5]:
# Step 4: Get headers of table 
t_headers = []
for th in gdp_table.find_all("th"):
    # remove any newlines and extra spaces from left and right
    t_headers.append(th.text.replace('\n', ' ').strip())
t_headers

['Postal Code', 'Borough', 'Neighborhood']

In [6]:
# Get all the rows of table
table_data = []
for tr in gdp_table.tbody.find_all("tr"): # find all tr's from table's tbody
    t_row = {}

    for td, th in zip(tr.find_all("td"), t_headers): 
        t_row[th] = td.text.replace('\n', '').strip()
    table_data.append(t_row)

In [7]:
import pandas as pd
import numpy as np 
df = pd.DataFrame(data=table_data[1:], columns = t_headers)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [19]:
df = df[df['Borough'] != 'Not assigned']

In [20]:
df.shape

(103, 3)

# Part 2: Retrieve Lat and Long for each Neighborhood

In [10]:
import requests # library to handle requests
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: done

# All requested packages already installed.



In [11]:
lat_long_df = pd.read_csv('/resources/labs/Geospatial_Coordinates.csv')

In [21]:
result_df = pd.merge(df, #left dataframe
                 lat_long_df, #right dataframe
                 left_on = 'Postal Code', #left_on specifies merge column name(s) in left dataframe
                 right_on = 'Postal Code', #right_on specifies merge column name(s) in right dataframe
                 how = 'inner') #how can be one of: inner, left, right, outer

In [22]:
result_df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
